In [1]:
import os
os.chdir('../..')
!ls

product_impacts


In [2]:
import os.path
import pandas as pd
from skimpy import skim
import numpy as np
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc

sns.set()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
os.environ["RAY_ENABLE_MAC_LARGE_OBJECT_STORE"] = "1"

In [3]:
# enter path here
data_dir = '../../SFS/openfoodfacts/all/'
prefix = 'eng/'

In [4]:
pred_nn = pd.read_csv(f'{data_dir}/predictions/{prefix}predictions_nn.csv')
pred_rf = pd.read_csv(f'{data_dir}/predictions/{prefix}predictions_rf.csv')

/Users/shruti/anaconda3/envs/ct_map/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
pred = pred_nn
pred.shape

(755225, 8)

In [9]:
pred['parentcategory'].value_counts()

Cereals and Cereal Products             145532
Sugar, Preserves and Confectionery       88643
Vegetables, Potatoes                     77635
Miscellaneous                            68189
Milk and Milk Products                   65317
Non-Alcoholic Beverages                  62807
Meat and Meat Products                   60810
Alcoholic Beverages                      50525
Fruit                                    27451
Not Food                                 27296
Nuts and Seeds                           26688
Savoury Snacks                           25754
Fish and Fish Dishes                     15942
Fat Spreads                               7054
Dietary Supplements                       2197
Eggs and Egg Dishes                       2092
Commercial Toddlers Foods and Drinks       669
Artificial Sweeteners                      624
Name: parentcategory, dtype: int64

In [6]:
pred['subfoodgroup'].value_counts()

Savoury sauces pickles gravies & condiments                      55338
Alcoholic Beverages                                              50525
Chocolate confectionery                                          47820
Biscuits                                                         44158
Not Food                                                         27296
Sugar confectionery                                              26812
Nuts and Seeds                                                   26688
Other vegetables                                                 25913
Savoury Snacks                                                   25754
Other chicken and turkey                                         22397
Buns cakes and pastries                                          19257
Pasta                                                            18612
Other cheese                                                     16912
Other fruit not canned                                           16321
White 

In [14]:
pred.sample(100).sort_values(by=['parentcategory', 'subfoodgroup'])[['product_name_en', 'ingredients_text_en', 'countries_en',
                                                                     'parentcategory', 'mainfoodgroup', 'subfoodgroup',
                                                                     'parentcategory_prob', 'subfoodgroup_prob']].reset_index(drop=True)

,product_name_en,ingredients_text_en,countries_en,parentcategory,mainfoodgroup,subfoodgroup,parentcategory_prob,subfoodgroup_prob
0,Somat Pulver,NaN,Germany,Alcoholic Beverages,Alcoholic Beverages,Alcoholic Beverages,0.993003,NaN
1,Aminodrink Sour Cherry,NaN,Germany,Alcoholic Beverages,Alcoholic Beverages,Alcoholic Beverages,0.913288,NaN
2,Mandorle Affettare,NaN,Italy,Alcoholic Beverages,Alcoholic Beverages,Alcoholic Beverages,0.974939,NaN
3,Nature amandes,NaN,France,Alcoholic Beverages,Alcoholic Beverages,Alcoholic Beverages,0.538317,NaN
4,Naturell Mineralwasser,NaN,Germany,Alcoholic Beverages,Alcoholic Beverages,Alcoholic Beverages,0.989959,NaN
5,Cough and mucus,NaN,United States,Alcoholic Beverages,Alcoholic Beverages,Alcoholic Beverages,0.690470,NaN
6,"Sorbet Abricot, Back D'un Litre, Marque Carte ...",ILe-650ge v.rww.LovelceCream.com,France,Alcoholic Beverages,Alcoholic Beverages,Alcoholic Beverages,0.870225,NaN
7,Wafer coconut,"Sugar, Wheat Flour, Vegetable Fats in Varying ...",Italy,Cereals and Cereal Products,Biscuits,Biscuits,0.977702,0.999865
8,"2 brioche chocolate popovers, 2 brioche","Wheat flour, canola oil, sugar, eggs, egg yolk...",United States,Cereals and Cereal Products,Biscuits,Biscuits,0.997554,0.836108
9,Sesame bars with honey,NaN,Switzerland,Cereals and Cereal Products,Biscuits,Biscuits,0.631302,0.998937


In [15]:
skim(pred_nn)

╭───────────────────────────────────── skimpy summary ──────────────────────────────────────╮
│          Data Summary                Data Types                                           │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                    │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                    │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                    │
│ │ Number of rows    │ 755225 │ │ object      │ 24    │                                    │
│ │ Number of columns │ 35     │ │ float64     │ 10    │                                    │
│ └───────────────────┴────────┘ │ int64       │ 1     │                                    │
│                                └─────────────┴───────┘                                    │
│                                          number                                           │
│ ┏━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┓  │
│ ┃        ┃ missin ┃ comple ┃ mean   ┃ sd     ┃ p0    ┃ p25    ┃ p75   ┃ p100   ┃ hist  ┃  │
│ ┃        ┃ g      ┃ te     ┃        ┃        ┃       ┃        ┃       ┃        ┃       ┃  │
│ ┃        ┃        ┃ rate   ┃        ┃        ┃       ┃        ┃       ┃        ┃       ┃  │
│ ┡━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━┩  │
│ │ produc │      0 │      1 │ 200000 │ 130000 │ 29000 │ 290000 │ 29000 │ 290000 │   ▄   │  │
│ │ t_id   │        │        │ 000000 │ 000000 │  0000 │ 000000 │ 00000 │ 000000 │   █   │  │
│ │        │        │        │    000 │    000 │       │     00 │ 00000 │    000 │       │  │
│ │ energy │ 110000 │   0.86 │ 150000 │ 120000 │     0 │    440 │  1700 │  1e+17 │   █   │  │
│ │ _100g  │        │        │ 000000 │ 000000 │       │        │       │        │       │  │
│ │        │        │        │        │    000 │       │        │       │        │       │  │
│ │ fat_10 │ 110000 │   0.85 │   2500 │ 200000 │     0 │    0.5 │    20 │ 160000 │   █   │  │
│ │ 0g     │        │        │        │      0 │       │        │       │   0000 │       │  │
│ │ satura │ 150000 │    0.8 │   1000 │ 770000 │     0 │      0 │   6.7 │ 600000 │   █   │  │
│ │ ted-fa │        │        │        │        │       │        │       │    000 │       │  │
│ │ t_100g │        │        │        │        │       │        │       │        │       │  │
│ │ carboh │ 110000 │   0.86 │ 150000 │ 120000 │    -1 │    7.3 │    58 │ 100000 │   █   │  │
│ │ ydrate │        │        │    000 │ 000000 │       │        │       │ 000000 │       │  │
│ │ s_100g │        │        │        │        │       │        │       │    000 │       │  │
│ │ fiber_ │ 310000 │   0.59 │ 150000 │ 100000 │     0 │      0 │     4 │ 670000 │   █   │  │
│ │ 100g   │        │        │      0 │   0000 │       │        │       │ 000000 │       │  │
│ │ protei │ 110000 │   0.85 │ 100000 │ 830000 │     0 │    1.2 │    11 │ 670000 │   █   │  │
│ │ ns_100 │        │        │    000 │  00000 │       │        │       │ 000000 │       │  │
│ │ g      │        │        │        │        │       │        │       │     00 │       │  │
│ │ sugars │ 130000 │   0.83 │     19 │   2600 │     0 │    1.2 │    23 │ 210000 │   █   │  │
│ │ _100g  │        │        │        │        │       │        │       │      0 │       │  │
│ │ salt_1 │ 170000 │   0.77 │ 140000 │ 100000 │     0 │   0.11 │   1.3 │ 800000 │   █   │  │
│ │ 00g    │        │        │    000 │ 000000 │       │        │       │ 000000 │       │  │
│ │        │        │        │        │        │       │        │       │     00 │       │  │
│ │ parent │      0 │      1 │    0.9 │   0.16 │  0.13 │   0.88 │     1 │      1 │   ▁▁▁ │  │
│ │ catego │        │        │        │        │       │        │       │        │   █   │  │
│ │ ry_pro │        │        │        │        │       │        │       │        │       │  │
│ │ b      │        │        │        │        │    

In [16]:
skim(pred_rf)

╭───────────────────────────────────── skimpy summary ──────────────────────────────────────╮
│          Data Summary                Data Types                                           │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                    │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                    │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                    │
│ │ Number of rows    │ 755225 │ │ object      │ 24    │                                    │
│ │ Number of columns │ 35     │ │ float64     │ 10    │                                    │
│ └───────────────────┴────────┘ │ int64       │ 1     │                                    │
│                                └─────────────┴───────┘                                    │
│                                          number                                           │
│ ┏━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┓  │
│ ┃        ┃ missin ┃ comple ┃ mean   ┃ sd     ┃ p0    ┃ p25    ┃ p75   ┃ p100   ┃ hist  ┃  │
│ ┃        ┃ g      ┃ te     ┃        ┃        ┃       ┃        ┃       ┃        ┃       ┃  │
│ ┃        ┃        ┃ rate   ┃        ┃        ┃       ┃        ┃       ┃        ┃       ┃  │
│ ┡━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━┩  │
│ │ produc │      0 │      1 │ 200000 │ 130000 │ 29000 │ 290000 │ 29000 │ 290000 │   ▄   │  │
│ │ t_id   │        │        │ 000000 │ 000000 │  0000 │ 000000 │ 00000 │ 000000 │   █   │  │
│ │        │        │        │    000 │    000 │       │     00 │ 00000 │    000 │       │  │
│ │ energy │ 110000 │   0.86 │ 150000 │ 120000 │     0 │    440 │  1700 │  1e+17 │   █   │  │
│ │ _100g  │        │        │ 000000 │ 000000 │       │        │       │        │       │  │
│ │        │        │        │        │    000 │       │        │       │        │       │  │
│ │ fat_10 │ 110000 │   0.85 │   2500 │ 200000 │     0 │    0.5 │    20 │ 160000 │   █   │  │
│ │ 0g     │        │        │        │      0 │       │        │       │   0000 │       │  │
│ │ satura │ 150000 │    0.8 │   1000 │ 770000 │     0 │      0 │   6.7 │ 600000 │   █   │  │
│ │ ted-fa │        │        │        │        │       │        │       │    000 │       │  │
│ │ t_100g │        │        │        │        │       │        │       │        │       │  │
│ │ carboh │ 110000 │   0.86 │ 150000 │ 120000 │    -1 │    7.3 │    58 │ 100000 │   █   │  │
│ │ ydrate │        │        │    000 │ 000000 │       │        │       │ 000000 │       │  │
│ │ s_100g │        │        │        │        │       │        │       │    000 │       │  │
│ │ fiber_ │ 310000 │   0.59 │ 150000 │ 100000 │     0 │      0 │     4 │ 670000 │   █   │  │
│ │ 100g   │        │        │      0 │   0000 │       │        │       │ 000000 │       │  │
│ │ protei │ 110000 │   0.85 │ 100000 │ 830000 │     0 │    1.2 │    11 │ 670000 │   █   │  │
│ │ ns_100 │        │        │    000 │  00000 │       │        │       │ 000000 │       │  │
│ │ g      │        │        │        │        │       │        │       │     00 │       │  │
│ │ sugars │ 130000 │   0.83 │     19 │   2600 │     0 │    1.2 │    23 │ 210000 │   █   │  │
│ │ _100g  │        │        │        │        │       │        │       │      0 │       │  │
│ │ salt_1 │ 170000 │   0.77 │ 140000 │ 100000 │     0 │   0.11 │   1.3 │ 800000 │   █   │  │
│ │ 00g    │        │        │    000 │ 000000 │       │        │       │ 000000 │       │  │
│ │        │        │        │        │        │       │        │       │     00 │       │  │
│ │ parent │      0 │      1 │   0.38 │   0.16 │ 0.094 │   0.25 │  0.48 │      1 │ ▅█▅▃▁ │  │
│ │ catego │        │        │        │        │       │        │       │        │       │  │
│ │ ry_pro │        │        │        │        │       │        │       │        │       │  │
│ │ b      │        │        │        │        │    

In [17]:
pred_compare = pred_nn.rename(columns={
    'parentcategory': 'parentcategory_nn',
    'mainfoodgroup': 'mainfoodgroup_nn',
    'subfoodgroup': 'subfoodgroup_nn',
    'parentcategory_prob': 'parentcategory_prob_nn',
    'subfoodgroup_prob': 'subfoodgroup_prob_nn'
}).merge(pred_rf.rename(columns={
    'parentcategory': 'parentcategory_rf',
    'mainfoodgroup': 'mainfoodgroup_rf',
    'subfoodgroup': 'subfoodgroup_rf',
    'parentcategory_prob': 'parentcategory_prob_rf',
    'subfoodgroup_prob': 'subfoodgroup_prob_rf'
}))

In [18]:
# rf confidence remains noisy and low in all cases, but maybe its even lower when both models dont agree, maybe more tuning is needed for rf?

In [19]:
# nn is quite confident when both models agree on predicted parent category
skim(pred_compare[pred_compare['parentcategory_nn']==pred_compare['parentcategory_rf']])

╭───────────────────────────────────── skimpy summary ──────────────────────────────────────╮
│          Data Summary                Data Types                                           │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                    │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                    │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                    │
│ │ Number of rows    │ 599777 │ │ object      │ 27    │                                    │
│ │ Number of columns │ 40     │ │ float64     │ 12    │                                    │
│ └───────────────────┴────────┘ │ int64       │ 1     │                                    │
│                                └─────────────┴───────┘                                    │
│                                          number                                           │
│ ┏━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┓  │
│ ┃        ┃ missin ┃ comple ┃ mean   ┃ sd     ┃ p0    ┃ p25    ┃ p75   ┃ p100   ┃ hist  ┃  │
│ ┃        ┃ g      ┃ te     ┃        ┃        ┃       ┃        ┃       ┃        ┃       ┃  │
│ ┃        ┃        ┃ rate   ┃        ┃        ┃       ┃        ┃       ┃        ┃       ┃  │
│ ┡━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━┩  │
│ │ produc │      0 │      1 │ 210000 │ 120000 │ 29000 │ 290000 │ 29000 │ 290000 │   ▄   │  │
│ │ t_id   │        │        │ 000000 │ 000000 │  0000 │ 000000 │ 00000 │ 000000 │   █   │  │
│ │        │        │        │    000 │    000 │       │     00 │ 00000 │    000 │       │  │
│ │ energy │  80000 │   0.87 │ 190000 │ 140000 │     0 │    430 │  1700 │  1e+17 │   █   │  │
│ │ _100g  │        │        │ 000000 │ 000000 │       │        │       │        │       │  │
│ │        │        │        │        │    000 │       │        │       │        │       │  │
│ │ fat_10 │  83000 │   0.86 │   3100 │ 220000 │     0 │    0.5 │    19 │ 160000 │   █   │  │
│ │ 0g     │        │        │        │      0 │       │        │       │   0000 │       │  │
│ │ satura │ 120000 │    0.8 │   1200 │ 860000 │     0 │      0 │   6.7 │ 600000 │   █   │  │
│ │ ted-fa │        │        │        │        │       │        │       │    000 │       │  │
│ │ t_100g │        │        │        │        │       │        │       │        │       │  │
│ │ carboh │  82000 │   0.86 │ 190000 │ 140000 │    -1 │    7.1 │    58 │ 100000 │   █   │  │
│ │ ydrate │        │        │    000 │ 000000 │       │        │       │ 000000 │       │  │
│ │ s_100g │        │        │        │        │       │        │       │    000 │       │  │
│ │ fiber_ │ 240000 │   0.61 │ 180000 │ 110000 │     0 │      0 │   3.8 │ 670000 │   █   │  │
│ │ 100g   │        │        │      0 │   0000 │       │        │       │ 000000 │       │  │
│ │ protei │  83000 │   0.86 │ 130000 │ 930000 │     0 │    1.2 │    11 │ 670000 │   █   │  │
│ │ ns_100 │        │        │    000 │  00000 │       │        │       │ 000000 │       │  │
│ │ g      │        │        │        │        │       │        │       │     00 │       │  │
│ │ sugars │ 100000 │   0.83 │     19 │   2900 │     0 │    1.2 │    23 │ 210000 │   █   │  │
│ │ _100g  │        │        │        │        │       │        │       │      0 │       │  │
│ │ salt_1 │ 130000 │   0.78 │ 170000 │ 120000 │     0 │   0.11 │   1.3 │ 800000 │   █   │  │
│ │ 00g    │        │        │    000 │ 000000 │       │        │       │ 000000 │       │  │
│ │        │        │        │        │        │       │        │       │     00 │       │  │
│ │ parent │      0 │      1 │   0.94 │   0.12 │  0.17 │   0.96 │     1 │      1 │     ▁ │  │
│ │ catego │        │        │        │        │       │        │       │        │   █   │  │
│ │ ry_pro │        │        │        │        │       │        │       │        │       │  │
│ │ b_     │        │        │        │        │    

In [20]:
# nn is quite confident (lev 0 and lev 2) when both models agree on lev 0 and lev 2 classes
skim(pred_compare[(pred_compare['parentcategory_nn']==pred_compare['parentcategory_rf'])
             & (pred_compare['subfoodgroup_nn']==pred_compare['subfoodgroup_rf'])])

╭───────────────────────────────────── skimpy summary ──────────────────────────────────────╮
│          Data Summary                Data Types                                           │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                    │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                    │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                    │
│ │ Number of rows    │ 534129 │ │ object      │ 27    │                                    │
│ │ Number of columns │ 40     │ │ float64     │ 12    │                                    │
│ └───────────────────┴────────┘ │ int64       │ 1     │                                    │
│                                └─────────────┴───────┘                                    │
│                                          number                                           │
│ ┏━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┓  │
│ ┃        ┃ missin ┃ comple ┃ mean   ┃ sd     ┃ p0    ┃ p25    ┃ p75   ┃ p100   ┃ hist  ┃  │
│ ┃        ┃ g      ┃ te     ┃        ┃        ┃       ┃        ┃       ┃        ┃       ┃  │
│ ┃        ┃        ┃ rate   ┃        ┃        ┃       ┃        ┃       ┃        ┃       ┃  │
│ ┡━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━┩  │
│ │ produc │      0 │      1 │ 210000 │ 120000 │ 29000 │ 290000 │ 29000 │ 290000 │   ▄   │  │
│ │ t_id   │        │        │ 000000 │ 000000 │  0000 │ 000000 │ 00000 │ 000000 │   █   │  │
│ │        │        │        │    000 │    000 │       │     00 │ 00000 │    000 │       │  │
│ │ energy │  73000 │   0.86 │ 220000 │ 150000 │     0 │    440 │  1700 │  1e+17 │   █   │  │
│ │ _100g  │        │        │ 000000 │ 000000 │       │        │       │        │       │  │
│ │        │        │        │        │    000 │       │        │       │        │       │  │
│ │ fat_10 │  76000 │   0.86 │   3500 │ 240000 │     0 │    0.5 │    20 │ 160000 │   █   │  │
│ │ 0g     │        │        │        │      0 │       │        │       │   0000 │       │  │
│ │ satura │ 110000 │    0.8 │   1400 │ 920000 │     0 │      0 │   6.8 │ 600000 │   █   │  │
│ │ ted-fa │        │        │        │        │       │        │       │    000 │       │  │
│ │ t_100g │        │        │        │        │       │        │       │        │       │  │
│ │ carboh │  75000 │   0.86 │     34 │     78 │    -1 │      7 │    58 │  12000 │   █   │  │
│ │ ydrate │        │        │        │        │       │        │       │        │       │  │
│ │ s_100g │        │        │        │        │       │        │       │        │       │  │
│ │ fiber_ │ 210000 │    0.6 │    3.9 │    280 │     0 │      0 │   3.7 │ 160000 │   █   │  │
│ │ 100g   │        │        │        │        │       │        │       │        │       │  │
│ │ protei │  76000 │   0.86 │    9.1 │     56 │     0 │    1.3 │    11 │  14000 │   █   │  │
│ │ ns_100 │        │        │        │        │       │        │       │        │       │  │
│ │ g      │        │        │        │        │       │        │       │        │       │  │
│ │ sugars │  91000 │   0.83 │     20 │   3100 │     0 │    1.2 │    23 │ 210000 │   █   │  │
│ │ _100g  │        │        │        │        │       │        │       │      0 │       │  │
│ │ salt_1 │ 120000 │   0.78 │ 190000 │ 120000 │     0 │   0.12 │   1.4 │ 800000 │   █   │  │
│ │ 00g    │        │        │    000 │ 000000 │       │        │       │ 000000 │       │  │
│ │        │        │        │        │        │       │        │       │     00 │       │  │
│ │ parent │      0 │      1 │   0.94 │   0.12 │  0.17 │   0.96 │     1 │      1 │     ▁ │  │
│ │ catego │        │        │        │        │       │        │       │        │   █   │  │
│ │ ry_pro │        │        │        │        │       │        │       │        │       │  │
│ │ b_     │        │        │        │        │    

In [21]:
# nn confidence reduces (esp for lev 2) when both models agree on lev 0 but disagree on lev 2, these are explored in next cell
skim(pred_compare[(pred_compare['parentcategory_nn']==pred_compare['parentcategory_rf'])
             & (pred_compare['subfoodgroup_nn']!=pred_compare['subfoodgroup_rf'])])

╭───────────────────────────────────── skimpy summary ──────────────────────────────────────╮
│          Data Summary                Data Types                                           │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                    │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                    │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                    │
│ │ Number of rows    │ 65648  │ │ object      │ 27    │                                    │
│ │ Number of columns │ 40     │ │ float64     │ 12    │                                    │
│ └───────────────────┴────────┘ │ int64       │ 1     │                                    │
│                                └─────────────┴───────┘                                    │
│                                          number                                           │
│ ┏━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┓  │
│ ┃        ┃ missin ┃ comple ┃ mean   ┃ sd     ┃ p0    ┃ p25    ┃ p75   ┃ p100   ┃ hist  ┃  │
│ ┃        ┃ g      ┃ te     ┃        ┃        ┃       ┃        ┃       ┃        ┃       ┃  │
│ ┃        ┃        ┃ rate   ┃        ┃        ┃       ┃        ┃       ┃        ┃       ┃  │
│ ┡━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━┩  │
│ │ produc │      0 │      1 │ 210000 │ 120000 │ 29000 │ 290000 │ 29000 │ 290000 │   ▄   │  │
│ │ t_id   │        │        │ 000000 │ 000000 │  0000 │ 000000 │ 00000 │ 000000 │   █   │  │
│ │        │        │        │    000 │    000 │       │     00 │ 00000 │    000 │       │  │
│ │ energy │   6800 │    0.9 │   1100 │   4100 │     0 │    370 │  1600 │ 510000 │   █   │  │
│ │ _100g  │        │        │        │        │       │        │       │        │       │  │
│ │ fat_10 │   7200 │   0.89 │    9.9 │     44 │     0 │    0.1 │    14 │   5300 │   █   │  │
│ │ 0g     │        │        │        │        │       │        │       │        │       │  │
│ │ satura │  12000 │   0.82 │    3.7 │     48 │     0 │      0 │   4.2 │  11000 │   █   │  │
│ │ ted-fa │        │        │        │        │       │        │       │        │       │  │
│ │ t_100g │        │        │        │        │       │        │       │        │       │  │
│ │ carboh │   7000 │   0.89 │ 170000 │ 410000 │     0 │    7.5 │    60 │ 100000 │   █   │  │
│ │ ydrate │        │        │   0000 │ 000000 │       │        │       │ 000000 │       │  │
│ │ s_100g │        │        │        │        │       │        │       │    000 │       │  │
│ │ fiber_ │  24000 │   0.64 │ 160000 │ 330000 │     0 │      0 │     4 │ 670000 │   █   │  │
│ │ 100g   │        │        │     00 │   0000 │       │        │       │ 000000 │       │  │
│ │ protei │   7200 │   0.89 │ 110000 │ 280000 │     0 │      1 │    11 │ 670000 │   █   │  │
│ │ ns_100 │        │        │   0000 │ 000000 │       │        │       │ 000000 │       │  │
│ │ g      │        │        │        │        │       │        │       │     00 │       │  │
│ │ sugars │   9000 │   0.86 │     13 │     24 │     0 │    1.2 │    18 │   2000 │   █   │  │
│ │ _100g  │        │        │        │        │       │        │       │        │       │  │
│ │ salt_1 │  13000 │    0.8 │    3.3 │    220 │     0 │    0.1 │   1.2 │  50000 │   █   │  │
│ │ 00g    │        │        │        │        │       │        │       │        │       │  │
│ │ parent │      0 │      1 │   0.93 │   0.14 │  0.18 │   0.93 │     1 │      1 │    ▁▁ │  │
│ │ catego │        │        │        │        │       │        │       │        │   █   │  │
│ │ ry_pro │        │        │        │        │       │        │       │        │       │  │
│ │ b_     │        │        │        │        │       │        │       │        │       │  │
│ │ subfoo │      0 │      1 │   0.76 │   0.19 │  0.21 │    0.6 │  0.95 │      1 │  ▁▄▄▄ │  │
│ │ dgroup │        │        │        │        │    

In [22]:
pred_compare[(pred_compare['parentcategory_nn']==pred_compare['parentcategory_rf'])
             & (pred_compare['subfoodgroup_nn']!=pred_compare['subfoodgroup_rf'])].sample(100)[[
    'product_name_en', 'ingredients_text_en', 
    'parentcategory_nn', 'subfoodgroup_nn',
    'parentcategory_prob_nn', 'subfoodgroup_prob_nn', 
    'parentcategory_rf', 'subfoodgroup_rf',
    'parentcategory_prob_rf', 'subfoodgroup_prob_rf']].reset_index(drop=True)

,product_name_en,ingredients_text_en,parentcategory_nn,subfoodgroup_nn,parentcategory_prob_nn,subfoodgroup_prob_nn,parentcategory_rf,subfoodgroup_rf,parentcategory_prob_rf,subfoodgroup_prob_rf
0,Wholemeal Sandwich,NaN,Cereals and Cereal Products,Wholemeal bread,0.882369,0.999374,Cereals and Cereal Products,Other cereals,0.140162,0.132815
1,Oh! Ricey Rice Noodles,NaN,Cereals and Cereal Products,Rice,0.999134,0.664963,Cereals and Cereal Products,Pasta,0.306120,0.382175
2,Roasted favourite beans,NaN,"Vegetables, Potatoes",Beans and pulses (including ready meal & homem...,0.981302,0.935474,"Vegetables, Potatoes",Green beans,0.429796,0.349423
3,"French vanilla instant pudding & pie filling, ...","Sugar, modified food starch, dextrose, disodiu...",Cereals and Cereal Products,Cereal based milk puddings,0.791127,0.467439,Cereals and Cereal Products,Buns cakes and pastries,0.317051,0.327664
4,New england clam chowder,"Half & half (milk, cream), clams (clams, clam ...",Fish and Fish Dishes,Other white fish,0.992907,0.955576,Fish and Fish Dishes,Shellfish,0.351336,0.335106
5,Organic Whole Grain Waffles Flax,"(that we are proud of!) water, organic whole w...",Cereals and Cereal Products,Buns cakes and pastries,0.999304,0.926219,Cereals and Cereal Products,Biscuits,0.520013,0.229058
6,Kip Grill Worst Natural,NaN,Meat and Meat Products,Burgers and kebabs purchased,0.684702,0.638436,Meat and Meat Products,Beef and veal,0.151921,0.156944
7,Fluffy pancake sandwich caramel,NaN,Cereals and Cereal Products,Other cereal based puddings,0.733775,0.995039,Cereals and Cereal Products,Buns cakes and pastries,0.245813,0.271333
8,"Original daisy brownies, original","Sugar, bleached enriched wheat flour (wheat fl...",Cereals and Cereal Products,Buns cakes and pastries,0.995444,0.500884,Cereals and Cereal Products,Biscuits,0.441989,0.292876
9,Wavy dark russet chips,NaN,"Vegetables, Potatoes",Other vegetables,0.979862,0.523944,"Vegetables, Potatoes",Chips purchased including takeaway,0.161419,0.252162


In [23]:
# nn confidence reduces (for both lev 0 and lev 2) when both models don't agree on parent category, these are explored in next cell
skim(pred_compare[pred_compare['parentcategory_nn']!=pred_compare['parentcategory_rf']])

╭───────────────────────────────────── skimpy summary ──────────────────────────────────────╮
│          Data Summary                Data Types                                           │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                    │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                    │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                    │
│ │ Number of rows    │ 155448 │ │ object      │ 27    │                                    │
│ │ Number of columns │ 40     │ │ float64     │ 12    │                                    │
│ └───────────────────┴────────┘ │ int64       │ 1     │                                    │
│                                └─────────────┴───────┘                                    │
│                                          number                                           │
│ ┏━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┓  │
│ ┃        ┃ missin ┃ comple ┃ mean   ┃ sd     ┃ p0    ┃ p25    ┃ p75   ┃ p100   ┃ hist  ┃  │
│ ┃        ┃ g      ┃ te     ┃        ┃        ┃       ┃        ┃       ┃        ┃       ┃  │
│ ┃        ┃        ┃ rate   ┃        ┃        ┃       ┃        ┃       ┃        ┃       ┃  │
│ ┡━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━┩  │
│ │ produc │      0 │      1 │ 170000 │ 130000 │ 29000 │ 290000 │ 29000 │ 290000 │   ▆   │  │
│ │ t_id   │        │        │ 000000 │ 000000 │ 00000 │ 000000 │ 00000 │ 000000 │   █   │  │
│ │        │        │        │    000 │    000 │       │     00 │ 00000 │    000 │       │  │
│ │ energy │  26000 │   0.83 │   1400 │  10000 │     0 │    490 │  1800 │ 190000 │   █   │  │
│ │ _100g  │        │        │        │        │       │        │       │      0 │       │  │
│ │ fat_10 │  27000 │   0.83 │     14 │     47 │     0 │   0.72 │    21 │   5300 │   █   │  │
│ │ 0g     │        │        │        │        │       │        │       │        │       │  │
│ │ satura │  34000 │   0.78 │    4.9 │     17 │     0 │      0 │   6.7 │   2000 │   █   │  │
│ │ ted-fa │        │        │        │        │       │        │       │        │       │  │
│ │ t_100g │        │        │        │        │       │        │       │        │       │  │
│ │ carboh │  26000 │   0.83 │     37 │     92 │     0 │    9.6 │    58 │  13000 │   █   │  │
│ │ ydrate │        │        │        │        │       │        │       │        │       │  │
│ │ s_100g │        │        │        │        │       │        │       │        │       │  │
│ │ fiber_ │  72000 │   0.54 │    4.4 │     51 │     0 │   0.05 │     5 │  14000 │   █   │  │
│ │ 100g   │        │        │        │        │       │        │       │        │       │  │
│ │ protei │  27000 │   0.83 │     11 │     52 │     0 │    1.2 │    11 │   5100 │   █   │  │
│ │ ns_100 │        │        │        │        │       │        │       │        │       │  │
│ │ g      │        │        │        │        │       │        │       │        │       │  │
│ │ sugars │  30000 │    0.8 │     16 │     32 │     0 │    1.5 │    24 │   3400 │   █   │  │
│ │ _100g  │        │        │        │        │       │        │       │        │       │  │
│ │ salt_1 │  43000 │   0.72 │    4.1 │    310 │     0 │   0.12 │   1.3 │  95000 │   █   │  │
│ │ 00g    │        │        │        │        │       │        │       │        │       │  │
│ │ parent │      0 │      1 │   0.77 │    0.2 │  0.13 │    0.6 │  0.96 │      1 │  ▁▃▃▃ │  │
│ │ catego │        │        │        │        │       │        │       │        │   █   │  │
│ │ ry_pro │        │        │        │        │       │        │       │        │       │  │
│ │ b_     │        │        │        │        │       │        │       │        │       │  │
│ │ subfoo │  34000 │   0.78 │   0.89 │   0.16 │  0.22 │   0.84 │     1 │      1 │   ▁▁▁ │  │
│ │ dgroup │        │        │        │        │    

In [24]:
pred_compare[(pred_compare['parentcategory_nn']!=pred_compare['parentcategory_rf'])].sample(100)[[
    'product_name_en', 'ingredients_text_en', 
    'parentcategory_nn', 'subfoodgroup_nn',
    'parentcategory_prob_nn', 'subfoodgroup_prob_nn', 
    'parentcategory_rf', 'subfoodgroup_rf',
    'parentcategory_prob_rf', 'subfoodgroup_prob_rf']].reset_index(drop=True)

,product_name_en,ingredients_text_en,parentcategory_nn,subfoodgroup_nn,parentcategory_prob_nn,subfoodgroup_prob_nn,parentcategory_rf,subfoodgroup_rf,parentcategory_prob_rf,subfoodgroup_prob_rf
0,Borscht,"Water, beets, sugar, salt & citric acid.","Vegetables, Potatoes",Other vegetable products and dishes,0.339579,0.687694,Miscellaneous,Savoury sauces pickles gravies & condiments,0.179892,0.690654
1,Platinium hydro whey,NaN,Miscellaneous,Nutrition powders and drinks,0.928933,0.968708,Milk and Milk Products,Other cheese,0.174584,0.258688
2,Good Energy For Me,NaN,Non-Alcoholic Beverages,Bottled water still or carbonated,0.485315,0.999260,Alcoholic Beverages,Alcoholic Beverages,0.240566,NaN
3,Sandwich Saumon fumé Radis blanc et wasabi,NaN,"Vegetables, Potatoes",Peas,0.570611,0.856006,Milk and Milk Products,Other cheese,0.132994,0.522070
4,Infusion thym citronne,NaN,Non-Alcoholic Beverages,Herbal tea (made up),0.915631,0.530137,Alcoholic Beverages,Alcoholic Beverages,0.226377,NaN
5,bloody point,NaN,Alcoholic Beverages,Alcoholic Beverages,0.946383,NaN,Not Food,Not Food,0.166210,NaN
6,Sweetened dried cranberries,"Cranberries, sugar, sunflower oil.",Fruit,Other fruit not canned,0.998281,0.988603,Cereals and Cereal Products,Buns cakes and pastries,0.269136,0.244827
7,Peanut Butter Dark Choc and Rock Salt Granola,"Wholegrain Oats (45%), Peanut Butter (14%), Ri...",Nuts and Seeds,Nuts and Seeds,0.537630,NaN,Cereals and Cereal Products,High fibre breakfast cereals,0.404593,0.374734
8,Thon Germon Huile Olive,NaN,Fat Spreads,Other cooking fats and oils not polyunsaturated,0.903820,0.999253,Alcoholic Beverages,Alcoholic Beverages,0.293004,NaN
9,Black olive paste,NaN,Fat Spreads,Other cooking fats and oils not polyunsaturated,0.844667,0.997830,Not Food,Not Food,0.141762,NaN
